In [ ]:
!pip install ctransformers
!pip install lancedb
!pip install langchain
!pip install sentence-transformers

In [7]:
from langchain_community.vectorstores import LanceDB
from langchain.embeddings import HuggingFaceEmbeddings

import lancedb

In [18]:
embedder = HuggingFaceEmbeddings(
    model_name='sentence-transformers/all-MiniLM-L6-v2',
    model_kwargs={'device': 'cpu'}
)

In [19]:
lancedb_instance = lancedb.connect("/tmp/lancedb")

lancedb_table = lancedb_instance.create_table(
    "my_table",
    data=[
        {
            "vector": embedder.embed_query("Fupi is a small giraffe."),
            "text": "Fupi is a small giraffe.",
            "id": "1",
        }
    ],
    mode="overwrite",
)

document_searcher = LanceDB(lancedb_table, embedder)

# LanceDB.__init__()
# 2 required positional arguments: 'connection' and 'embedding'

# def __init__(
#     connection: Any,
#     embedding: Embeddings,
#     vector_key: Optional[str]='vector',
#     id_key: Optional[str]='id',
#     text_key: Optional[str]='text'
# )

# connection should be an instance of lancedb.db.LanceTable

In [ ]:
from langchain.llms import CTransformers
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# config = {
#     'max_new_tokens': 256,
#     'repetition_penalty': 1.1
# }

llama = CTransformers(
    model='TheBloke/Llama-2-7B-GGML',
    # config=config,
    callbacks=[StreamingStdOutCallbackHandler()]
)

In [21]:
from langchain.chains import RetrievalQA

question_answering = RetrievalQA.from_chain_type(
    llm=llama,
    chain_type="stuff",
    retriever=document_searcher.as_retriever()
  )

In [22]:
query = "What kind of animal is Fupi?"

question_answering.run(query)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 A cow (which has nothing to do with Fupi)

Answer: A giraffe.

\begin{blockquote}

Giraffes are very intelligent animals, and they have a great sense of humor as well

([Answer](https://www.thefuntimes.com/2015/09/whats-kind-of-animal-is-fupi.html))
\end{blockquote}

' A cow (which has nothing to do with Fupi)\n\nAnswer: A giraffe.\n\n\\begin{blockquote}\n\nGiraffes are very intelligent animals, and they have a great sense of humor as well\n\n([Answer](https://www.thefuntimes.com/2015/09/whats-kind-of-animal-is-fupi.html))\n\\end{blockquote}'